In [ ]:
import tempfile
from enum import Enum, unique
from pathlib import Path
from typing import List, Optional, Tuple, Union

import requests
from torchvision.datasets.utils import check_integrity

from health_multimodal.image import ImageInferenceEngine
from health_multimodal.image.data.transforms import create_chest_xray_transform_for_inference
from health_multimodal.image.model.pretrained import get_biovil_t_image_encoder
from health_multimodal.text.utils import BertEncoderType, get_bert_inference
from health_multimodal.vlp.inference_engine import ImageTextInferenceEngine

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
RESIZE = 512
CENTER_CROP_SIZE = 512

In [ ]:
def _get_vlp_inference_engine() -> ImageTextInferenceEngine:
    image_inference = ImageInferenceEngine(
        image_model=get_biovil_t_image_encoder(),
        transform=create_chest_xray_transform_for_inference(resize=RESIZE, center_crop_size=CENTER_CROP_SIZE),
    )
    img_txt_inference = ImageTextInferenceEngine(
        image_inference_engine=image_inference,
        text_inference_engine=get_bert_inference(BertEncoderType.BIOVIL_T_BERT),
    )
    return img_txt_inference

In [ ]:
def get_cosine_similarity(image_path, search_query):
    '''
    Returns cosine similarity between resized image and search query
    '''
    img_txt_inference = _get_vlp_inference_engine()
    cosine_similarity = img_txt_inference.get_similarity_score_from_raw_data(
                image_path=image_path, query_text=search_query
            )
    return cosine_similarity

In [ ]:
def show_image(image_path):
    '''
    Displays jpg image
    '''    
    image = Image.open(image_path)
    
    plt.imshow(image, cmap=plt.cm.gray)
    plt.axis('off')  
    plt.show()

In [ ]:
pneumonia_pos = "/opt/gpudata/mimic-cxr/jpg/p10/p10011365/s53459647/f6fccc21-ded29731-2a7419a6-961566fe-710630d3.jpg"

show_image(pneumonia_pos)
print(get_cosine_similarity(Path(pneumonia_pos), "Pneumonia seen"))
print(get_cosine_similarity(Path(pneumonia_pos), "Findings consistent with pneumonia"))
print(get_cosine_similarity(Path(pneumonia_pos), "Findings suggesting pneumonia"))
print(get_cosine_similarity(Path(pneumonia_pos), "This opacity can represent pneumonia"))
print(get_cosine_similarity(Path(pneumonia_pos), "Findings are most compatible with pneumonia"))



In [ ]:
file_path = "/opt/gpudata/mimic-cxr/notes/p10/p10011365/s53459647.txt"
with open(file_path, 'r') as file:
    contents = file.read()

print(contents)

In [ ]:
pneumonia_pos = "/opt/gpudata/mimic-cxr/jpg/p10/p10011365/s53459647/f6fccc21-ded29731-2a7419a6-961566fe-710630d3.jpg"

show_image(pneumonia_pos)
print(get_cosine_similarity(Path(pneumonia_pos), "Pneumonia is not seen"))
print(get_cosine_similarity(Path(pneumonia_pos), "Findings not consistent with pneumonia"))
print(get_cosine_similarity(Path(pneumonia_pos), "Findings not suggesting pneumonia"))
print(get_cosine_similarity(Path(pneumonia_pos), "Findings are not compatible with pneumonia"))

In [ ]:
pneumonia_neg = "/opt/gpudata/mimic-cxr/jpg/p10/p10000980/s50985099/6ad03ed1-97ee17ee-9cf8b320-f7011003-cd93b42d.jpg"
show_image(pneumonia_neg)

print(get_cosine_similarity(Path(pneumonia_neg), "Pneumonia seen"))
print(get_cosine_similarity(Path(pneumonia_neg), "Findings consistent with pneumonia"))
print(get_cosine_similarity(Path(pneumonia_neg), "Findings suggesting pneumonia"))
print(get_cosine_similarity(Path(pneumonia_neg), "This opacity can represent pneumonia"))
print(get_cosine_similarity(Path(pneumonia_neg), "Findings are most compatible with pneumonia"))

In [ ]:
file_path = "/opt/gpudata/mimic-cxr/notes/p10/p10000980/s50985099.txt"
with open(file_path, 'r') as file:
    contents = file.read()

print(contents)

In [ ]:
pneumonia_neg = "/opt/gpudata/mimic-cxr/jpg/p10/p10000980/s50985099/6ad03ed1-97ee17ee-9cf8b320-f7011003-cd93b42d.jpg"
show_image(pneumonia_neg)

print(get_cosine_similarity(Path(pneumonia_neg), "Pneumonia is not seen"))
print(get_cosine_similarity(Path(pneumonia_neg), "Findings not consistent with pneumonia"))
print(get_cosine_similarity(Path(pneumonia_neg), "Findings not suggesting pneumonia"))
print(get_cosine_similarity(Path(pneumonia_neg), "Findings are not compatible with pneumonia"))